In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import warnings
import tensorflow as tf
import seaborn as sns
import sklearn
import random
import math
import time


from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
from tqdm import tqdm
from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from tensorflow.keras import optimizers


from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
from methods import *

%matplotlib inline
warnings.filterwarnings('ignore')

SEED = 0
def set_seed(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(SEED)
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)

# Appeler la fonction pour fixer le seed
set_seed(SEED)

In [2]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD004.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR', 'farB','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 120
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(61249, 27) (41214, 26) (248, 1)


In [3]:
def model_lstm_1layer(input_shape, nodes_per_layer, dropout, activation):
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(LSTM(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(256))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=Adam(learning_rate=0.001))
    # model.save_weights(weights_file)

    return model

In [4]:
%%time
results_all = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # 0.20	[64]	0.3	tanh	32	25
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.3
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all = pd.concat([results_all, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all.to_csv('results/all/fd004.csv')

(51538, 40, 17) (51538, 1) (248, 40, 17)
Epoch 1/20
1611/1611 [==============================] - 14s 8ms/step - loss: 762.7661 - val_loss: 328.0673
Epoch 2/20
1611/1611 [==============================] - 12s 8ms/step - loss: 294.9990 - val_loss: 232.8337
Epoch 3/20
1611/1611 [==============================] - 12s 7ms/step - loss: 249.8761 - val_loss: 213.0574
Epoch 4/20
1611/1611 [==============================] - 14s 8ms/step - loss: 231.6885 - val_loss: 207.1930
Epoch 5/20
1611/1611 [==============================] - 12s 7ms/step - loss: 215.2335 - val_loss: 189.2900
Epoch 6/20
1611/1611 [==============================] - 12s 7ms/step - loss: 200.3619 - val_loss: 187.0126
Epoch 7/20
1611/1611 [==============================] - 12s 7ms/step - loss: 189.2251 - val_loss: 180.8434
Epoch 8/20
1611/1611 [==============================] - 11s 7ms/step - loss: 182.3082 - val_loss: 169.7781
Epoch 9/20
1611/1611 [==============================] - 11s 7ms/step - loss: 175.0479 - val_loss: 163.9

OSError: [Errno 89] Operation canceled: 'results/all/fd004.csv'

In [5]:
results_all

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,12.551669,0.0,741.853794,0.0,157.544388,0.0,[64],0.2,tanh,32,40,560.656871


### FD002 <a class="anchor" id="fd002"></a>

In [6]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [7]:
%%time
results_all002 = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.2
    sequence_length = 40
    epochs = 20
    nodes_per_layer = [32]
    dropout = 0.1
    activation = 'tanh'
    batch_size = 128
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout, activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all002 = pd.concat([results_all002, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all002.to_csv('results/all/fd002.csv')

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 4s 10ms/step - loss: 2300.0369 - val_loss: 1411.3716
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 552.4333 - val_loss: 282.0428
Epoch 3/20
341/341 [==============================] - 3s 8ms/step - loss: 301.3049 - val_loss: 205.2548
Epoch 4/20
341/341 [==============================] - 3s 8ms/step - loss: 257.1705 - val_loss: 195.6782
Epoch 5/20
341/341 [==============================] - 3s 8ms/step - loss: 243.2023 - val_loss: 182.1209
Epoch 6/20
341/341 [==============================] - 3s 8ms/step - loss: 237.6594 - val_loss: 189.3480
Epoch 7/20
341/341 [==============================] - 3s 8ms/step - loss: 233.1519 - val_loss: 176.0869
Epoch 8/20
341/341 [==============================] - 3s 8ms/step - loss: 229.4595 - val_loss: 192.5895
Epoch 9/20
341/341 [==============================] - 3s 9ms/step - loss: 222.5657 - val_loss: 189.8916
Epoch 10/20
341/341 

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 4s 10ms/step - loss: 2267.1296 - val_loss: 1253.7079
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 469.4091 - val_loss: 206.8166
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 269.0750 - val_loss: 196.0345
Epoch 4/20
341/341 [==============================] - 3s 9ms/step - loss: 258.0416 - val_loss: 184.8317
Epoch 5/20
341/341 [==============================] - 4s 10ms/step - loss: 239.6045 - val_loss: 223.4978
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 236.3609 - val_loss: 191.5826
Epoch 7/20
341/341 [==============================] - 3s 9ms/step - loss: 232.2512 - val_loss: 194.0178
Epoch 8/20
341/341 [==============================] - 3s 9ms/step - loss: 230.5434 - val_loss: 195.0752
Epoch 9/20
341/341 [==============================] - 3s 9ms/step - loss: 226.8534 - val_loss: 193.9557
Epoch 10/20
341/341

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 4s 9ms/step - loss: 2347.6218 - val_loss: 1665.2920
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 639.2714 - val_loss: 271.6960
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 295.7878 - val_loss: 219.6517
Epoch 4/20
341/341 [==============================] - 3s 9ms/step - loss: 269.8353 - val_loss: 196.1009
Epoch 5/20
341/341 [==============================] - 3s 9ms/step - loss: 255.6487 - val_loss: 189.8894
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 245.1190 - val_loss: 186.4090
Epoch 7/20
341/341 [==============================] - 3s 9ms/step - loss: 239.0444 - val_loss: 189.2510
Epoch 8/20
341/341 [==============================] - 3s 9ms/step - loss: 234.4229 - val_loss: 198.7714
Epoch 9/20
341/341 [==============================] - 3s 8ms/step - loss: 228.7659 - val_loss: 179.4258
Epoch 10/20
341/341 [

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 4s 11ms/step - loss: 2280.4766 - val_loss: 1257.2867
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 560.7480 - val_loss: 355.6381
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 321.4266 - val_loss: 247.3982
Epoch 4/20
341/341 [==============================] - 3s 9ms/step - loss: 254.9767 - val_loss: 293.5551
Epoch 5/20
341/341 [==============================] - 3s 9ms/step - loss: 248.4174 - val_loss: 187.3849
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 237.5092 - val_loss: 199.7161
Epoch 7/20
341/341 [==============================] - 3s 9ms/step - loss: 231.6589 - val_loss: 180.8135
Epoch 8/20
341/341 [==============================] - 3s 9ms/step - loss: 229.7975 - val_loss: 179.4874
Epoch 9/20
341/341 [==============================] - 3s 10ms/step - loss: 229.6983 - val_loss: 189.3564
Epoch 10/20
341/341

(43619, 40, 16) (43619, 1) (259, 40, 16)
Epoch 1/20
341/341 [==============================] - 4s 10ms/step - loss: 2291.9856 - val_loss: 1213.2866
Epoch 2/20
341/341 [==============================] - 3s 9ms/step - loss: 498.0440 - val_loss: 221.4526
Epoch 3/20
341/341 [==============================] - 3s 9ms/step - loss: 273.8747 - val_loss: 216.0187
Epoch 4/20
341/341 [==============================] - 3s 10ms/step - loss: 257.3766 - val_loss: 222.0951
Epoch 5/20
341/341 [==============================] - 3s 9ms/step - loss: 243.6585 - val_loss: 204.7415
Epoch 6/20
341/341 [==============================] - 3s 9ms/step - loss: 237.9813 - val_loss: 201.8890
Epoch 7/20
341/341 [==============================] - 3s 9ms/step - loss: 232.4224 - val_loss: 230.1372
Epoch 8/20
341/341 [==============================] - 3s 9ms/step - loss: 231.1712 - val_loss: 195.0920
Epoch 9/20
341/341 [==============================] - 3s 10ms/step - loss: 225.9395 - val_loss: 205.4482
Epoch 10/20
341/34

In [8]:
results_all002

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,13.102145,0.0,825.334576,0.0,171.666199,0.0,[32],0.1,tanh,128,40,60.258299
1,13.228334,0.0,850.169654,0.0,174.988831,0.0,[32],0.1,tanh,128,40,60.961792
2,13.108823,0.0,824.245323,0.0,171.841248,0.0,[32],0.1,tanh,128,40,60.634674
3,13.295064,0.0,888.831273,0.0,176.758713,0.0,[32],0.1,tanh,128,40,63.824795
4,13.305796,0.0,685.164746,0.0,177.044189,0.0,[32],0.1,tanh,128,40,65.198323


### FD003 <a class="anchor" id="fd003"></a>

In [9]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD002.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc', 'epr','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(53759, 27) (33991, 26) (259, 1)


In [10]:
%%time
results_all003 = pd.DataFrame()
for SEED in range(5):  
    tf.random.set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.1
    sequence_length = 35
    epochs = 20
    nodes_per_layer = [64]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 32
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all003 = pd.concat([results_all003, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all003.to_csv('results/all/fd003.csv')

(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 12s 8ms/step - loss: 906.4077 - val_loss: 297.4064
Epoch 2/20
1404/1404 [==============================] - 10s 7ms/step - loss: 278.6239 - val_loss: 234.8844
Epoch 3/20
1404/1404 [==============================] - 10s 7ms/step - loss: 248.1741 - val_loss: 195.1236
Epoch 4/20
1404/1404 [==============================] - 10s 7ms/step - loss: 234.3593 - val_loss: 199.3235
Epoch 5/20
1404/1404 [==============================] - 10s 7ms/step - loss: 223.6005 - val_loss: 174.3159
Epoch 6/20
1404/1404 [==============================] - 9s 7ms/step - loss: 213.5243 - val_loss: 167.1391
Epoch 7/20
1404/1404 [==============================] - 9s 7ms/step - loss: 208.4235 - val_loss: 174.1897
Epoch 8/20
1404/1404 [==============================] - 9s 6ms/step - loss: 201.9832 - val_loss: 174.3790
Epoch 9/20
1404/1404 [==============================] - 9s 7ms/step - loss: 196.2110 - val_loss: 172.3315


(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 11s 7ms/step - loss: 830.6611 - val_loss: 235.3775
Epoch 2/20
1404/1404 [==============================] - 9s 6ms/step - loss: 298.7138 - val_loss: 215.1112
Epoch 3/20
1404/1404 [==============================] - 9s 6ms/step - loss: 257.1817 - val_loss: 185.4624
Epoch 4/20
1404/1404 [==============================] - 9s 6ms/step - loss: 246.8819 - val_loss: 181.2589
Epoch 5/20
1404/1404 [==============================] - 9s 7ms/step - loss: 232.8512 - val_loss: 187.8301
Epoch 6/20
1404/1404 [==============================] - 9s 6ms/step - loss: 223.6786 - val_loss: 175.8104
Epoch 7/20
1404/1404 [==============================] - 9s 6ms/step - loss: 218.0415 - val_loss: 168.6120
Epoch 8/20
1404/1404 [==============================] - 9s 6ms/step - loss: 208.3120 - val_loss: 187.1732
Epoch 9/20
1404/1404 [==============================] - 10s 7ms/step - loss: 204.9613 - val_loss: 176.7870
Epo

(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 10s 6ms/step - loss: 928.9767 - val_loss: 240.5249
Epoch 2/20
1404/1404 [==============================] - 9s 6ms/step - loss: 274.2685 - val_loss: 202.2992
Epoch 3/20
1404/1404 [==============================] - 9s 6ms/step - loss: 240.3042 - val_loss: 182.1285
Epoch 4/20
1404/1404 [==============================] - 9s 6ms/step - loss: 225.5036 - val_loss: 221.9795
Epoch 5/20
1404/1404 [==============================] - 10s 7ms/step - loss: 219.4206 - val_loss: 181.7211
Epoch 6/20
1404/1404 [==============================] - 9s 6ms/step - loss: 210.7127 - val_loss: 192.3570
Epoch 7/20
1404/1404 [==============================] - 9s 6ms/step - loss: 207.0006 - val_loss: 181.0675
Epoch 8/20
1404/1404 [==============================] - 9s 6ms/step - loss: 199.9305 - val_loss: 188.1906
Epoch 9/20
1404/1404 [==============================] - 9s 6ms/step - loss: 195.2570 - val_loss: 171.5017
Epo

(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 10s 7ms/step - loss: 877.4888 - val_loss: 323.8827
Epoch 2/20
1404/1404 [==============================] - 9s 6ms/step - loss: 290.3685 - val_loss: 216.2555
Epoch 3/20
1404/1404 [==============================] - 9s 6ms/step - loss: 252.0514 - val_loss: 206.6399
Epoch 4/20
1404/1404 [==============================] - 9s 6ms/step - loss: 238.7656 - val_loss: 181.9300
Epoch 5/20
1404/1404 [==============================] - 9s 7ms/step - loss: 225.9218 - val_loss: 189.2038
Epoch 6/20
1404/1404 [==============================] - 10s 7ms/step - loss: 216.4854 - val_loss: 188.2886
Epoch 7/20
1404/1404 [==============================] - 9s 7ms/step - loss: 209.0519 - val_loss: 174.5823
Epoch 8/20
1404/1404 [==============================] - 9s 6ms/step - loss: 202.9315 - val_loss: 177.9902
Epoch 9/20
1404/1404 [==============================] - 9s 6ms/step - loss: 197.5704 - val_loss: 161.3453
Epo

(44919, 35, 16) (44919, 1) (259, 35, 16)
Epoch 1/20
1404/1404 [==============================] - 10s 7ms/step - loss: 838.2953 - val_loss: 222.6092
Epoch 2/20
1404/1404 [==============================] - 9s 7ms/step - loss: 271.8162 - val_loss: 193.0789
Epoch 3/20
1404/1404 [==============================] - 10s 7ms/step - loss: 244.3942 - val_loss: 183.2736
Epoch 4/20
1404/1404 [==============================] - 9s 6ms/step - loss: 232.2839 - val_loss: 225.5190
Epoch 5/20
1404/1404 [==============================] - 9s 6ms/step - loss: 222.7054 - val_loss: 188.9413
Epoch 6/20
1404/1404 [==============================] - 9s 6ms/step - loss: 215.0832 - val_loss: 165.5778
Epoch 7/20
1404/1404 [==============================] - 10s 7ms/step - loss: 208.8830 - val_loss: 173.6259
Epoch 8/20
1404/1404 [==============================] - 9s 7ms/step - loss: 201.7564 - val_loss: 165.5113
Epoch 9/20
1404/1404 [==============================] - 9s 6ms/step - loss: 195.4309 - val_loss: 170.5264
Ep

In [11]:
results_all003

,RMSE,std_RMSE,S_score,std_S_score,MSE,std_MSE,nodes,dropout,activation,batch_size,TW,time
0,12.363102,0.0,667.250275,0.0,152.846313,0.0,[64],0.2,tanh,32,35,190.864901
1,12.683775,0.0,668.448743,0.0,160.878143,0.0,[64],0.2,tanh,32,35,183.677209
2,12.981874,0.0,740.183392,0.0,168.529068,0.0,[64],0.2,tanh,32,35,182.398291
3,12.763592,0.0,795.465533,0.0,162.909286,0.0,[64],0.2,tanh,32,35,182.180160
4,12.724188,0.0,630.904330,0.0,161.904953,0.0,[64],0.2,tanh,32,35,185.120741


### FD001

In [12]:
# Load data
# Load data and preprocess
train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                    'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']
remaining_sensors = ['T24','T30','T50', 'P15', 'P30','Nf','Nc','Ps30','phi',
                    'NRf','NRc','BPR','htBleed','W31','W32']
drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

rul_piecewise = 125
train['RUL'].clip(upper=rul_piecewise, inplace=True)

(20631, 27) (13096, 26) (100, 1)


In [13]:
%%time
results_all001 = pd.DataFrame()
for SEED in range(5):  
    set_seed(SEED)
    mse = []
    R2_val = []
    RMSE = []
    score_val = []
    
    # parameter's sample
    # weights_file = "weights_file_lstm_optimalmodel_all.h5"
    alpha = 0.1
    sequence_length = 30
    epochs = 20
    nodes_per_layer = [128]
    dropout = 0.2
    activation = 'tanh'
    batch_size = 64
    remaining_sensors = remaining_sensors
    # create model
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_lstm_1layer(input_shape, nodes_per_layer[0], dropout,
                             activation)
    
    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    test_rul = rul_piecewise_fct(y_test,rul_piecewise)
    print(train_array.shape, label_array.shape, test_array.shape)
            
    # Model fitting
    with tf.device('/device:GPU:0'):
        start_time = time.time()
        model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))
        weights_file = model.get_weights()
        model.set_weights(weights_file)  # reset optimizer and node weights before every training iteration
        history = model.fit(train_array, label_array,
                                validation_data=(test_array, test_rul),
                                epochs=epochs,
                                batch_size=batch_size,
                                # callbacks=[cb],
                                verbose=1)
        end_time = time.time()
        training_time = end_time - start_time
        mse.append(history.history['val_loss'][-1])

        y_hat_val_split = model.predict(test_array)
        R2_val.append(r2_score(test_rul, y_hat_val_split))
        RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
        score_val.append(compute_s_score(test_rul, y_hat_val_split))
            
        
    #  append results
    d = {'RMSE' :np.mean(RMSE), 'std_RMSE' :np.std(RMSE),
         'S_score' :np.mean(score_val), 'std_S_score' :np.std(score_val),
         'MSE':np.mean(mse), 'std_MSE':np.std(mse),
         'nodes':str(nodes_per_layer), 'dropout':dropout, 
         'activation':activation, 'batch_size':batch_size, 'TW' : sequence_length,
         'time':training_time}

#     results = results.append(pd.DataFrame(d, index=[0]), ignore_index=True)
    results_all001 = pd.concat([results_all001, pd.DataFrame(d, index=[0])], ignore_index=True)
    results_all001.to_csv('results/all/fd001.csv')

(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 6s 20ms/step - loss: 1945.8270 - val_loss: 1011.8328
Epoch 2/20
278/278 [==============================] - 5s 19ms/step - loss: 557.1711 - val_loss: 283.6362
Epoch 3/20
278/278 [==============================] - 5s 18ms/step - loss: 336.0125 - val_loss: 402.8049
Epoch 4/20
278/278 [==============================] - 5s 19ms/step - loss: 256.9118 - val_loss: 234.6717
Epoch 5/20
278/278 [==============================] - 6s 22ms/step - loss: 240.9247 - val_loss: 188.8959
Epoch 6/20
278/278 [==============================] - 5s 17ms/step - loss: 215.9529 - val_loss: 246.9015
Epoch 7/20
278/278 [==============================] - 5s 18ms/step - loss: 212.4774 - val_loss: 200.7240
Epoch 8/20
278/278 [==============================] - 5s 16ms/step - loss: 196.8468 - val_loss: 287.7190
Epoch 9/20
278/278 [==============================] - 5s 18ms/step - loss: 198.1949 - val_loss: 179.0840
Epoch 10/20


(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 6s 19ms/step - loss: 1952.8248 - val_loss: 989.5231
Epoch 2/20
278/278 [==============================] - 5s 17ms/step - loss: 608.6642 - val_loss: 457.7419
Epoch 3/20
278/278 [==============================] - 5s 17ms/step - loss: 358.4405 - val_loss: 682.9811
Epoch 4/20
278/278 [==============================] - 5s 17ms/step - loss: 251.7239 - val_loss: 341.1137
Epoch 5/20
278/278 [==============================] - 5s 17ms/step - loss: 211.5727 - val_loss: 217.0099
Epoch 6/20
278/278 [==============================] - 5s 18ms/step - loss: 201.9030 - val_loss: 207.4494
Epoch 7/20
278/278 [==============================] - 5s 17ms/step - loss: 192.0596 - val_loss: 207.3164
Epoch 8/20
278/278 [==============================] - 5s 17ms/step - loss: 189.4069 - val_loss: 176.4110
Epoch 9/20
278/278 [==============================] - 5s 17ms/step - loss: 183.2050 - val_loss: 192.1585
Epoch 10/20
2

(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 5s 17ms/step - loss: 1969.4290 - val_loss: 1184.3573
Epoch 2/20
278/278 [==============================] - 6s 20ms/step - loss: 569.7576 - val_loss: 422.7188
Epoch 3/20
278/278 [==============================] - 5s 17ms/step - loss: 306.3215 - val_loss: 222.5002
Epoch 4/20
278/278 [==============================] - 5s 16ms/step - loss: 235.5519 - val_loss: 214.6935
Epoch 5/20
278/278 [==============================] - 5s 17ms/step - loss: 214.0097 - val_loss: 179.1317
Epoch 6/20
278/278 [==============================] - 5s 16ms/step - loss: 201.6206 - val_loss: 170.3593
Epoch 7/20
278/278 [==============================] - 5s 19ms/step - loss: 195.8453 - val_loss: 250.8557
Epoch 8/20
278/278 [==============================] - 4s 16ms/step - loss: 191.1101 - val_loss: 202.9509
Epoch 9/20
278/278 [==============================] - 5s 16ms/step - loss: 186.9187 - val_loss: 181.4738
Epoch 10/20


(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 6s 18ms/step - loss: 1981.1896 - val_loss: 917.8284
Epoch 2/20
278/278 [==============================] - 5s 17ms/step - loss: 589.6582 - val_loss: 387.9435
Epoch 3/20
278/278 [==============================] - 5s 18ms/step - loss: 354.6008 - val_loss: 215.8253
Epoch 4/20
278/278 [==============================] - 5s 18ms/step - loss: 249.8626 - val_loss: 274.0669
Epoch 5/20
278/278 [==============================] - 5s 18ms/step - loss: 218.3911 - val_loss: 303.3423
Epoch 6/20
278/278 [==============================] - 5s 17ms/step - loss: 212.5883 - val_loss: 251.4697
Epoch 7/20
278/278 [==============================] - 5s 17ms/step - loss: 202.4792 - val_loss: 616.0234
Epoch 8/20
278/278 [==============================] - 5s 17ms/step - loss: 196.4964 - val_loss: 172.3805
Epoch 9/20
278/278 [==============================] - 5s 17ms/step - loss: 184.9444 - val_loss: 233.0498
Epoch 10/20
2

(17731, 30, 15) (17731, 1) (100, 30, 15)
Epoch 1/20
278/278 [==============================] - 6s 19ms/step - loss: 2036.7043 - val_loss: 1121.9376
Epoch 2/20
278/278 [==============================] - 5s 17ms/step - loss: 604.8963 - val_loss: 474.9707
Epoch 3/20
278/278 [==============================] - 5s 17ms/step - loss: 448.0006 - val_loss: 532.3290
Epoch 4/20
278/278 [==============================] - 5s 17ms/step - loss: 276.0841 - val_loss: 333.5952
Epoch 5/20
278/278 [==============================] - 5s 17ms/step - loss: 227.4386 - val_loss: 538.3504
Epoch 6/20
278/278 [==============================] - 5s 17ms/step - loss: 220.6311 - val_loss: 231.7316
Epoch 7/20
278/278 [==============================] - 5s 17ms/step - loss: 198.7297 - val_loss: 169.9673
Epoch 8/20
278/278 [==============================] - 5s 17ms/step - loss: 197.7237 - val_loss: 208.0873
Epoch 9/20
278/278 [==============================] - 5s 17ms/step - loss: 189.4026 - val_loss: 173.8299
Epoch 10/20
